# 多数据源查询的自动路由切换

比如有2个知识库，`旅游景点知识库` 和 `动物知识库`，希望通过用户提示词，自动切换到相关知识库回答问题。

## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    # model="qwen2", 
    model="glm4", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size = 128
Settings.chunk_overlap = 10

CPU times: user 4.71 s, sys: 766 ms, total: 5.48 s
Wall time: 5.36 s


## 加载数据

### 景点数据

In [2]:
%%time

from llama_index.readers.web import TrafilaturaWebReader

items=[
    "颐和园",
    "北海公园",
    "故宫",
    # "八达岭长城",
    # "天坛公园",
    # "恭王府",
    # "圆明园",
    # "景山公园",
    # "中山公园",
    # "陶然亭公园",
]

documents_attractions = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents_attractions)

CPU times: user 1.25 s, sys: 59.2 ms, total: 1.31 s
Wall time: 1.6 s


3

### 动物数据

In [3]:
%%time

items=[
    "东北虎",
    "金丝猴",
    "大熊猫",
    # "扬子鳄",
    # "娃娃鱼",
    # "小熊猫",
]

documents_animals = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents_animals)

CPU times: user 380 ms, sys: 6.25 ms, total: 386 ms
Wall time: 587 ms


3

## 建立向量索引和查询引擎

### 景点

In [4]:
%%time

from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

# 创建索引

parser = SentenceSplitter(chunk_size=128,chunk_overlap=10)
nodes = parser.get_nodes_from_documents(documents_attractions)
vector_index_attractions = VectorStoreIndex(nodes)

len(nodes)

CPU times: user 3.27 s, sys: 473 ms, total: 3.75 s
Wall time: 54.9 s


545

In [5]:
%%time

# 创建检索器

vector_retriever_attractions = vector_index_attractions.as_retriever()

CPU times: user 34 μs, sys: 5 μs, total: 39 μs
Wall time: 41.2 μs


In [6]:
%%time

from llama_index.core.tools import RetrieverTool

vector_tool_attractions = RetrieverTool.from_defaults(
    retriever=vector_retriever_attractions,
    description=(
        "旅游景点相关问题，可以从这里检索具体信息"
    ),
)

CPU times: user 16 μs, sys: 2 μs, total: 18 μs
Wall time: 21 μs


### 动物

In [7]:
%%time

# 创建索引

parser = SentenceSplitter(chunk_size=128,chunk_overlap=10)
nodes = parser.get_nodes_from_documents(documents_animals)
vector_index_animails = VectorStoreIndex(nodes)

len(nodes)

CPU times: user 2.64 s, sys: 352 ms, total: 2.99 s
Wall time: 42.7 s


435

In [8]:
%%time

# 创建检索器

vector_retriever_animals = vector_index_animails.as_retriever()

CPU times: user 34 μs, sys: 4 μs, total: 38 μs
Wall time: 41.5 μs


In [9]:
%%time

from llama_index.core.tools import RetrieverTool

vector_tool_animals = RetrieverTool.from_defaults(
    retriever=vector_retriever_animals,
    description=(
        "动物相关问题，可以从这里检索具体信息"
    ),
)

CPU times: user 17 μs, sys: 2 μs, total: 19 μs
Wall time: 22.6 μs


### 路由检索器

In [10]:
%%time

from llama_index.core.retrievers import RouterRetriever

from llama_index.core.selectors import LLMSingleSelector

retriever = RouterRetriever(
    selector=LLMSingleSelector.from_defaults(),
    retriever_tools=[
        vector_tool_attractions,
        vector_tool_animals,
    ],
)

CPU times: user 394 μs, sys: 3.79 ms, total: 4.18 ms
Wall time: 3.85 ms


In [11]:
%%time

retriever.retrieve(
    "颐和园在哪里"
)

CPU times: user 99.8 ms, sys: 12.1 ms, total: 112 ms
Wall time: 3.34 s


[NodeWithScore(node=TextNode(id_='561000ff-4f6d-4432-8d39-d6c110654dcc', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://baike.baidu.com/item/颐和园', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='f6e5546f58c9fc5d1ad40561ca6c2b05d6d1257e6f030c1a8bab75777e9c0390'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='3c983a0b-d386-4e2f-a905-8f8f6e424584', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='994ac7a04fdf52b0204cfb450624d5a98ee22b5b768555df61ddc271374c32c7')}, text='颐和园，中国清朝时期皇家园林，前身为清漪园，位于北京市西郊海淀区新建宫门路19号。 [52] [72]其始建于清乾隆十五年（1750年），咸丰十年（1860年）被英法联军烧毁。', mimetype='text/plain', start_char_idx=0, end_char_idx=88, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.7068467872800152),
 NodeWithScore(node=TextNode(id_='fb741001-09d1-4307-8280-78458c401400', embedding=None, metadata

### 查询引擎

In [12]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True,
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

CPU times: user 134 μs, sys: 19 μs, total: 153 μs
Wall time: 156 μs


In [13]:
%%time

response = query_engine.query("颐和园在哪里")
response.print_response_stream()

位于北京市西郊海淀区新建宫门路19号。CPU times: user 64.9 ms, sys: 7.67 ms, total: 72.6 ms
Wall time: 2.19 s


In [14]:
%%time

response = query_engine.query("东北虎的体重是多少")
response.print_response_stream()

成年东北虎雄性体重平均为250千克。CPU times: user 57.5 ms, sys: 8.71 ms, total: 66.2 ms
Wall time: 1.67 s


### 局限性

In [15]:
%%time

response = query_engine.query("介绍下颐和园的历史")
response.print_response_stream()

颐和园，原名万寿山清漪园，位于北京西北郊的皇家园林区“三山五园”中。它是清代最后兴建的一座园林，具有悠久的历史背景。根据现存清代档案，我们可以了解到颐和园的建设工程从光绪十六年（1890年）底开始，至光绪二十年（1894年）结束。在这段时间内，工程的进度每5天就会上报一次，详细记录了从旧历初一至初五、初六至初十等不同时间段的进展情况。这些档案为我们提供了颐和园建设过程中的宝贵信息。CPU times: user 244 ms, sys: 57.1 ms, total: 301 ms
Wall time: 4.12 s


In [16]:
%%time

response = query_engine.query("介绍下东北虎")
response.print_response_stream()

东北虎是一种体型庞大、头大而圆的猫科动物。其前额上有数条黑色横纹，中间常被串通，形似“王”字，因此被誉为“丛林之王”。这种老虎的牙齿非常强大，通常有30颗。它们的四个虎爪非常锐利，使用时可以伸出，不用时则缩回爪鞘以避免行走时摩擦地面。

在朝鲜北部的山区，可能存在少量东北虎个体，但由于缺乏野外调查资料，具体情况尚不明确。东北虎的繁殖季节主要在冬季，发情交配期一般在11月至翌年2月份。在此期间，它们的叫声特别响亮，可以传播至2千米之外。CPU times: user 266 ms, sys: 69.7 ms, total: 335 ms
Wall time: 4.92 s


## 增加摘要索引的查询引擎

### 景点摘要

In [17]:
%%time

from llama_index.core import SummaryIndex

# 创建摘要索引

parser = SentenceSplitter(chunk_size=1024,chunk_overlap=20)
nodes = parser.get_nodes_from_documents(documents_attractions)
summary_index_attractions = SummaryIndex(nodes)

len(nodes)

CPU times: user 56.1 ms, sys: 3.7 ms, total: 59.8 ms
Wall time: 57.6 ms


57

In [18]:
%%time

# 创建索引器

summary_retriever_attractions = summary_index_attractions.as_retriever()

CPU times: user 97 μs, sys: 0 ns, total: 97 μs
Wall time: 101 μs


In [19]:
%%time

summary_tool_attractions = RetrieverTool.from_defaults(
    retriever=summary_retriever_attractions,
    description=(
        "旅游景点相关问题，回答概要和总体方面的问题。"
        "如果问题只需要更具体的上下文，请不要使用。"
    ),
)

CPU times: user 14 μs, sys: 0 ns, total: 14 μs
Wall time: 16.2 μs


### 动物摘要

In [20]:
%%time

# 创建摘要索引

parser = SentenceSplitter(chunk_size=1024,chunk_overlap=20)
nodes = parser.get_nodes_from_documents(documents_animals)
summary_index_animals = SummaryIndex(nodes)

len(nodes)

CPU times: user 51.9 ms, sys: 0 ns, total: 51.9 ms
Wall time: 50.8 ms


44

In [21]:
%%time

# 创建索引器

summary_retriever_animals = summary_index_animals.as_retriever()

CPU times: user 25 μs, sys: 3 μs, total: 28 μs
Wall time: 30.8 μs


In [22]:
%%time

summary_tool_animals = RetrieverTool.from_defaults(
    retriever=summary_retriever_animals,
    description=(
        "动物相关问题，回答概要和总体方面的问题。"
        "如果问题只需要更具体的上下文，请不要使用。"
    ),
)

CPU times: user 12 μs, sys: 1 μs, total: 13 μs
Wall time: 16.5 μs


### 路由检索器

In [23]:
%%time

retriever = RouterRetriever(
    selector=LLMSingleSelector.from_defaults(),
    retriever_tools=[
        vector_tool_attractions,
        vector_tool_animals,
        summary_tool_attractions,
        summary_tool_animals,
    ],
)

CPU times: user 176 μs, sys: 25 μs, total: 201 μs
Wall time: 206 μs


In [24]:
%%time

retriever.retrieve(
    "颐和园在哪里"
)

CPU times: user 38.9 ms, sys: 160 μs, total: 39.1 ms
Wall time: 1.15 s


[NodeWithScore(node=TextNode(id_='561000ff-4f6d-4432-8d39-d6c110654dcc', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://baike.baidu.com/item/颐和园', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='f6e5546f58c9fc5d1ad40561ca6c2b05d6d1257e6f030c1a8bab75777e9c0390'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='3c983a0b-d386-4e2f-a905-8f8f6e424584', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='994ac7a04fdf52b0204cfb450624d5a98ee22b5b768555df61ddc271374c32c7')}, text='颐和园，中国清朝时期皇家园林，前身为清漪园，位于北京市西郊海淀区新建宫门路19号。 [52] [72]其始建于清乾隆十五年（1750年），咸丰十年（1860年）被英法联军烧毁。', mimetype='text/plain', start_char_idx=0, end_char_idx=88, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.7068467872800152),
 NodeWithScore(node=TextNode(id_='fb741001-09d1-4307-8280-78458c401400', embedding=None, metadata

In [25]:
%%time

nodes=retriever.retrieve(
    "颐和园的历史"
)

len(nodes)

CPU times: user 13.9 ms, sys: 4.14 ms, total: 18.1 ms
Wall time: 1.47 s


57

In [26]:
%%time

retriever.retrieve(
    "东北虎一般有多重"
)

CPU times: user 32.5 ms, sys: 3.55 ms, total: 36 ms
Wall time: 1.06 s


[NodeWithScore(node=TextNode(id_='6dcaed58-7dc5-4b01-b3a1-26d6abe914ea', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://baike.baidu.com/item/东北虎', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='665fb5ccb7f307f063b512fc4fbccf7be1d18d1eb61fa352b8789bd31b8a4c20'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='dd6b1ccc-6765-40bf-b6b6-e878980c2dd8', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='ff3ce8bd3c4cbf4c93ba3c2dc494bd47c75ba2df59a7f19c64415eaa5a419010'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c95c58fd-7160-4f1a-ad88-07be56b46eff', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='86792b80a2b6cbd90cb8b42cfb846cee581b4a9474375bae61cd065fe0f7dc15')}, text='[20]\n成年东北虎雄性体重平均为250千克，头体长约为2.3米；成年母虎平均体重约为170千克，体长约为2米，肩高1.1米左右，尾长1.3米左右。最大身长可达2.9米（含尾长）。', mimetype='text/plain', start_char_idx=3081, end_char_idx=3171, text

In [27]:
%%time

nodes=retriever.retrieve(
    "介绍下东北虎"
)
len(nodes)

CPU times: user 13.2 ms, sys: 4.04 ms, total: 17.3 ms
Wall time: 1.47 s


44

### 查询引擎

In [28]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True,
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

CPU times: user 162 μs, sys: 23 μs, total: 185 μs
Wall time: 188 μs


In [29]:
%%time

response = query_engine.query("颐和园在哪里")
response.print_response_stream()

位于北京市西郊海淀区新建宫门路19号。CPU times: user 71.8 ms, sys: 1.68 ms, total: 73.4 ms
Wall time: 1.68 s


In [30]:
%%time

response = query_engine.query("介绍下颐和园的历史")
response.print_response_stream()

颐和园位于北京市海淀区，是中国著名的皇家园林，其历史可追溯至清朝乾隆年间。最初名为“清漪园”，由乾隆皇帝于1750年左右下令建造，作为庆祝其母皇太后70寿辰的礼物。该园林占地约290公顷，以昆明湖为中心，建筑宏伟，景色秀丽。

然而，1860年英法联军入侵北京时，清漪园遭到严重破坏，大部分建筑被焚毁。1886年，清廷开始重建清漪园，两年后更名为颐和园。慈禧太后归政光绪后，将颐和园作为自己的“颐养冲和”之所。

1900年八国联军攻入北京，颐和园再次遭受浩劫，园藏文物被毁掠一空。但不久之后，慈禧太后回京后，不顾国家危难，再次修复颐和园，并大力充实园内陈设。

新中国成立后，特别是改革开放以来，颐和园得到了妥善的保护。目前，园内收藏有四万余件文物，涵盖了中国传世文物的所有门类。1998年，颐和园被联合国教科文组织列入《世界遗产名录》，成为全人类共同的文化财富。如今，它已成为国内外游客的热门旅游目的地，每年吸引着大量游客前来参观游览。CPU times: user 702 ms, sys: 127 ms, total: 829 ms
Wall time: 3min 5s


In [31]:
%%time

response = query_engine.query("东北虎一般有多重")
response.print_response_stream()

成年东北虎雄性体重平均为250千克。CPU times: user 67.3 ms, sys: 425 μs, total: 67.8 ms
Wall time: 1.79 s


In [32]:
%%time

response = query_engine.query("介绍下东北虎")
response.print_response_stream()

东北虎，学名Panthera tigris altaica，是猫科豹属的一种大型猛兽。主要分布在俄罗斯远东地区和中国东北地区。体型庞大，雄性体重约300公斤，雌性较轻。皮毛呈橙黄色，带有黑色条纹，尾巴末端为黑色。性格孤独，通常独自生活，繁殖季节寻找配偶。历史上数量减少，20世纪中叶以来有所恢复，但仍面临非法狩猎和贸易的威胁。东北虎是顶级掠食者，以大型哺乳动物为主食，如鹿、野猪等。繁殖期在冬季，怀孕期约105-110天，每产2-4仔。与已灭绝的里海虎有最近的亲缘关系，全球野生数量估计不足400只，被列为濒危物种。中国政府采取了一系列保护措施，包括建立自然保护区和加强执法力度。CPU times: user 486 ms, sys: 73.3 ms, total: 559 ms
Wall time: 1min 26s


## 结论

- 路由方式可以处理很多不同的检索器，可灵活增加数据来源
- SummaryIndex
    - 不适合过大的节点集合，因为它要在 `response_synthesis` 阶段，逐个将节点文档、提示词和之前的查询结果交由 LLM 处理，十分耗时
    - 适合用于节点数和内容少的数据实时摘要，或者对大文档数据源做预处理生成摘要